# Поиск и загрузка твиттов по ключевому слову
https://dev.twitter.com/rest/reference/get/search/tweets

отзывы зрителей нового эпизода “Звездных войн” https://habrahabr.ru/company/dca/blog/274027/

In [1]:
from IPython.core.display import clear_output
from urllib import unquote
import re

In [2]:
import twitter

In [3]:
# Go to http://dev.twitter.com/apps/new to create an app and get values
# for these credentials, which you'll need to provide in place of these
# empty string values that are defined as placeholders.
# See https://dev.twitter.com/docs/auth/oauth for more information 
# on Twitter's OAuth implementation.

CONSUMER_KEY = ''
CONSUMER_SECRET = ''
OAUTH_TOKEN = ''
OAUTH_TOKEN_SECRET = ''

In [4]:
auth = twitter.oauth.OAuth(OAUTH_TOKEN,
                           OAUTH_TOKEN_SECRET,
                           CONSUMER_KEY,
                           CONSUMER_SECRET)

twitter_api = twitter.Twitter(auth=auth, retry=True)

Задаем необходимые условия поиска

Искать твитты будем по следующим хештегам:
- #starwars
- #звездныевойны
- #ПробуждениеСилы
- #TheForceAwakens

По 1 000 уникальныт твиттов на каждый хештег

In [33]:
# По какому ключевому слову ищем
q = '#InternationalFriendshipDay'

# Сколько твиттов скачивать
count = 1000

# Сюда сохраняем твитты
all_tweets = []

Посылаем запросы на сервер твиттера пока не будет скачано заданое кол-во твиттов в переменной **count**

In [34]:
%%time
L = 0
while L<count:
    text_list = []
    
    # 1. Запрашиваем 100 твиттов
    try:
        search_results = twitter_api.search.tweets(q=q, count=100, lang='ru')
    except twitter.TwitterHTTPError as e:
            print e
            break
    
    statuses = search_results['statuses']
    
    # 1.1. Продолжаем запросы, если есть next_results
    for i in range(10):
        try:
            next_results = search_results['search_metadata']['next_results']
        except KeyError, e: # No more results when next_results doesn't exist
            print 'No more results'
            break

        # Create a dictionary from next_results, which has the following form:
        # ?max_id=313519052523986943&q=NCAA&include_entities=1
        kwargs = dict([kv.split('=') for kv in unquote(next_results[1:]).split("&") ])    

        try:
            search_results = twitter_api.search.tweets(**kwargs)
        except twitter.TwitterHTTPError as e:
            print e
            break
        
        statuses += search_results['statuses']

    # 2. Создаем список твиттов 
    for i in range(len(statuses)):
        text_list.append(statuses[i]['text'])

    # 3. Выбираем только уникальные твитты
    text_list = text_list + all_tweets
    text_set = set(text_list)
    all_tweets = list(text_set)
    L = len(all_tweets)
    
    clear_output(wait=True)
    print 'Tweets downloaded:', L
    


Tweets downloaded: 469


KeyboardInterrupt: 

In [24]:
for i in range(20):
    print all_tweets[i]

#InternationalFriendshipDay
даже поздравить некого
RT @weak_and_fallen: #InternationalFriendshipDay элиночка мы
RT @LarryIsRealMrr: когда тебе некого поздравлять с днем дружбы:
 #InternationalFriendshipDay https://t.co/o2ROsPqNm4
#InternationalFriendshipDay
@KOKC_HA_1D чмок, пупсик 💜
#InternationalFriendshipDay е мариначка
RT @etonezabvennaya: Хз с некоторыми друзья ли мы или не, но тех, кого отмечу - лав 💜
#InternationalFriendshipDay https://t.co/SruxsLv8I6
когда тебе некого поздравлять с днем дружбы:
 #InternationalFriendshipDay https://t.co/o2ROsPqNm4
#InternationalFriendshipDay
Лучшие❤
Люблю 😊😳😎 https://t.co/fHQWpXBn7k
RT @tgismylifemcr: #InternationalFriendshipDay
Васютка, с праздничком 
Смотри какую фоточку нашел, ты ее не видел еще 
Лав ю очень💜💜💜 https…
RT @kmadarina: #InternationalFriendshipDay
Всех лавки, кицы мои &lt;3
@Horinka3715 
@ya_ne_cupcakes 
@hannigramchild 
@_gallager6 https://t.co…
Конечно же ещё люблю Зили
Ты чудесная 💞 #InternationalFriendshipDay
Ксеня ❤😙
#Intern

In [19]:
search_results = twitter_api.search.tweets(q=q, count=100)
statuses = search_results['statuses']
len(statuses)

100

In [20]:
next_results = search_results['search_metadata']['next_results']

In [21]:
for i in range(20):
    print statuses[i]['user']['screen_name']
    print statuses[i]['text'], '\n'

SavageLindemann
RT @2pmwhitesides: love you chicken @SavageLindemann #InternationalFriendshipDay 

DavidBzatt
Wasn't it #InternationalFriendshipDay few weeks ago? I did a hilarious tweet about it that e/body loved. Well u don't deserve another, day. 

galexystyIes
@hugelovehes #InternationalFriendshipDay we just became friends and mutuals BUT YOURE SO SWEET AND KIND and your account always looks good 

Laura_SBerry
Happy #InternationalFriendshipDay to all my Tweeps and Twitter friends! 

cvarsalona
#InternationalFriendshipDay Happiness to all my connected edus around the globe. https://t.co/eMp1lFR0Y6 

taylorrkaye
RT @ArianatorIsland: since its #InternationalFriendshipDay let's appreciate the bond that Ariana &amp; Jones have. ☺️ https://t.co/zd7plTKKnH 

samsungsmargive
#InternationalFriendshipDay #TrumpSacrifices #3WordHillary

Pirates trade closer Mark Melancon to Nationals for two pitchers
Watch Here ➡ 

easeesivan
RT @smh_alyssaa: #InternationalFriendshipDay MUSERS FOR LIFE1!1!™ 

## Сохраняем скачанные твитты в файл для дальнейшего анализа

In [34]:
import pickle

In [35]:
pickle.dump(all_tweets, open('all_tweets.pkl', 'wb'), protocol=2)

Загрузка сохраненных данных

In [36]:
my_list2 = pickle.load(open('all_tweets.pkl', 'rb'))
len(my_list2)

10013